In [59]:
import pandas as pd
import torch.nn as nn
import pickle
import torch
from torchvision import models
from torchvision.models import detection, resnet50, ResNet50_Weights
import os
import numpy as np
import cv2
from torchvision import transforms
from sklearn.preprocessing import LabelEncoder
import scipy

import warnings
warnings.filterwarnings('ignore')

In [30]:
CONFIGS = {
    # determine the current device and based on that set the pin memory
    # flag
    "DEVICE": "cuda" if torch.cuda.is_available() else "cpu",
    # specify ImageNet mean and standard deviation
    "IMG_MEAN": [0.485, 0.456, 0.406],
    "IMG_STD": [0.229, 0.224, 0.225],
    "MC_DROPOUT_ENABLED": False,  # Switch to enable/disable MC Dropout for confidence score
    "NUM_DROPOUT_RUNS": 3,
    "CONFIDENCE_THRESHOLD": 0,
    "BIG_MODEL_IMG_SIZE": 320,
    "SMALL_MODEL_IMG_SIZE": 60,
    "MEAN_PRIOR": -15,
    "DATA_BASE_PATH": os.path.join('../..', 'rshiny', 'www', 'all_images'),
    "MODEL_PATH": 'traindatawithin2_2'
}

# Big model

## Model loading

In [31]:
class MultiHeadResNet_BigModel(nn.Module):
    def __init__(self, num_classes_prdtype, num_classes_weight, num_classes_halal, num_classes_healthy):
        super(MultiHeadResNet_BigModel, self).__init__()
        self.base_model = models.resnet50(weights=ResNet50_Weights.DEFAULT)
        num_ftrs = self.base_model.fc.in_features
        self.base_model.fc = nn.Identity()

        # Define custom fully connected layers for each prediction head
        self.fc_prdtype = nn.Linear(num_ftrs, num_classes_prdtype)
        self.fc_weight = nn.Linear(num_ftrs, num_classes_weight)
        self.fc_halal = nn.Linear(num_ftrs, num_classes_halal)
        self.fc_healthy = nn.Linear(num_ftrs, num_classes_healthy)
        self.fc_bbox = nn.Linear(num_ftrs, 4)

    def forward(self, x):
        x = self.base_model(x)
        prdtype = self.fc_prdtype(x)
        weight = self.fc_weight(x)
        halal = self.fc_halal(x)
        healthy = self.fc_healthy(x)
        box = self.fc_bbox(x)
        return prdtype, weight, halal, healthy, box

    
# load label encoder 
def load_label_encoder_big_model():
    le_prdtype = pickle.loads(open(os.path.join('../../NN_model/model_weights', CONFIGS['MODEL_PATH'], 'le_prdtype.pickle'), "rb").read())
    le_weight = pickle.loads(open(os.path.join('../../NN_model/model_weights', CONFIGS['MODEL_PATH'], 'le_weight.pickle'), "rb").read())
    le_halal = pickle.loads(open(os.path.join('../../NN_model/model_weights', CONFIGS['MODEL_PATH'], 'le_halal.pickle'), "rb").read())
    le_healthy = pickle.loads(open(os.path.join('../../NN_model/model_weights', CONFIGS['MODEL_PATH'], 'le_healthy.pickle'), "rb").read())
    
    return le_prdtype, le_weight, le_halal, le_healthy


le_prdtype, le_weight, le_halal, le_healthy = load_label_encoder_big_model()

# Load the trained MultiHeadResNet model
def load_model():
    # Verify the number of classes for each label
    num_classes_prdtype = len(le_prdtype.classes_)
    num_classes_weight = len(le_weight.classes_)
    num_classes_halal = len(le_halal.classes_)
    num_classes_healthy = len(le_healthy.classes_)
    # print(num_classes_prdtype)
    # print(num_classes_healthy)

    custom_resnet_model = MultiHeadResNet_BigModel(
        num_classes_prdtype=num_classes_prdtype,
        num_classes_weight=num_classes_weight,
        num_classes_halal=num_classes_halal,
        num_classes_healthy=num_classes_healthy
    )

    model_path = os.path.join('../../NN_model/model_weights', CONFIGS['MODEL_PATH'], 'multi_head_model.pth')
    # print("test1")
    if os.path.exists(model_path):
        custom_resnet_model.load_state_dict(torch.load(model_path, map_location=CONFIGS['DEVICE']))
    else:
        raise FileNotFoundError(f"Model file not found: {model_path}")
    # print("test2")
    custom_resnet_model.to(CONFIGS['DEVICE'])
    custom_resnet_model.eval()
    return custom_resnet_model

big_model = load_model()

/Users/liupeng/anaconda3/envs/py38/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


## Scoring on main imgs

In [32]:
main_imgs_results_big_model = pd.read_csv(os.path.join('../../NN_model/model_weights', CONFIGS['MODEL_PATH'], 'main_imgs_results_big_model.csv'))
main_imgs_results_big_model.head()

,Filename,CorrectTotalLabel,ProductType_BabyMilkPowder,ProductType_Babyfood,ProductType_BeehoonVermicelli,ProductType_BiscuitsCrackersCookies,ProductType_BreakfastCereals,ProductType_CannedBakedBeans,ProductType_CannedBeefOtherMeats,ProductType_CannedBraisedPeanuts,...,HalalStatus_199g,HalalStatus_299g,HalalStatus_399g,HalalStatus_Halal,HalalStatus_NonHalal,HealthStatus_Halal,HealthStatus_Healthy,HealthStatus_NonHalal,HealthStatus_NonHealthy,HealthStatus_Nonhealthy
0,IMG_2210_JPG.rf.daa853cfd9796f29b5468b3286e774...,BeehoonVermicelli_100-199g_NonHalal_NonHealthy,-2.053605,-1.263907,6.171493,-2.358087,-0.580795,-2.322378,-2.185837,-2.343666,...,-0.649461,-0.664399,-0.755979,-1.253223,3.437621,-1.617361,-1.657442,-1.930155,1.791324,-1.780641
1,output_frame_0591.png,SweetsChocolatesOthers_1-99g_NonHalal_NonHealthy,-1.727152,-1.615095,-1.283436,-1.931547,-1.844568,-1.934808,-1.677901,-1.697859,...,-0.681299,-0.766076,-0.764220,-0.757968,2.670682,-1.326687,-1.483171,-1.412153,1.950356,-1.376114
2,2023_8_11_11_39_48_208811_png.rf.d530c9e0d249e...,BabyMilkPowder_700-799g_Halal_Healthy,7.335651,-3.413930,-3.586535,-3.472556,-2.213640,-3.429164,-2.768110,-2.933477,...,-0.878268,-0.814173,-1.001764,2.501420,-1.337243,-1.802233,2.096767,-1.939219,-2.296831,-1.919529
3,20240219_12_2403.jpg,OtherSauceDressing_200-299g_NonHalal_NonHealthy,-1.844267,-1.954536,-1.982131,-1.682818,-2.304868,-1.458207,-1.998232,-1.795610,...,-0.667080,-0.813992,-0.664773,-0.554538,2.614168,-1.622760,-1.728462,-1.625550,1.808100,-1.463796
4,IMG_6900_jpeg.rf.530c3f64e20a3233c0421cd9aaa27...,OtherBakingNeeds_400-499g_NonHalal_NonHealthy,-0.894451,-1.714362,-2.250457,-2.714736,-0.854746,-2.529328,-2.300878,-2.260149,...,-0.815020,-0.652744,-0.703792,-0.591155,2.497566,-1.888847,-1.875948,-1.900468,1.467841,-1.778838


In [33]:
# Create a copy of the current column names to a list
new_columns = main_imgs_results_big_model.columns.tolist()

# Modify the first two elements
new_columns[0] = 'filepath'
new_columns[1] = 'label'

# Assign the modified list of column names back to the DataFrame
main_imgs_results_big_model.columns = new_columns


In [34]:
big_model_pred_col_name = [col for col in main_imgs_results_big_model.columns if col.startswith('ProductType_')]

def remove_prefix(text, prefix):
    if text.startswith(prefix):
        return text[len(prefix):]
    return text

big_model_pred_col_name = [remove_prefix(element, 'ProductType_') for element in big_model_pred_col_name]
# big_model_pred_col_name

In [35]:
main_imgs_results_big_model.head()

,filepath,label,ProductType_BabyMilkPowder,ProductType_Babyfood,ProductType_BeehoonVermicelli,ProductType_BiscuitsCrackersCookies,ProductType_BreakfastCereals,ProductType_CannedBakedBeans,ProductType_CannedBeefOtherMeats,ProductType_CannedBraisedPeanuts,...,HalalStatus_199g,HalalStatus_299g,HalalStatus_399g,HalalStatus_Halal,HalalStatus_NonHalal,HealthStatus_Halal,HealthStatus_Healthy,HealthStatus_NonHalal,HealthStatus_NonHealthy,HealthStatus_Nonhealthy
0,IMG_2210_JPG.rf.daa853cfd9796f29b5468b3286e774...,BeehoonVermicelli_100-199g_NonHalal_NonHealthy,-2.053605,-1.263907,6.171493,-2.358087,-0.580795,-2.322378,-2.185837,-2.343666,...,-0.649461,-0.664399,-0.755979,-1.253223,3.437621,-1.617361,-1.657442,-1.930155,1.791324,-1.780641
1,output_frame_0591.png,SweetsChocolatesOthers_1-99g_NonHalal_NonHealthy,-1.727152,-1.615095,-1.283436,-1.931547,-1.844568,-1.934808,-1.677901,-1.697859,...,-0.681299,-0.766076,-0.764220,-0.757968,2.670682,-1.326687,-1.483171,-1.412153,1.950356,-1.376114
2,2023_8_11_11_39_48_208811_png.rf.d530c9e0d249e...,BabyMilkPowder_700-799g_Halal_Healthy,7.335651,-3.413930,-3.586535,-3.472556,-2.213640,-3.429164,-2.768110,-2.933477,...,-0.878268,-0.814173,-1.001764,2.501420,-1.337243,-1.802233,2.096767,-1.939219,-2.296831,-1.919529
3,20240219_12_2403.jpg,OtherSauceDressing_200-299g_NonHalal_NonHealthy,-1.844267,-1.954536,-1.982131,-1.682818,-2.304868,-1.458207,-1.998232,-1.795610,...,-0.667080,-0.813992,-0.664773,-0.554538,2.614168,-1.622760,-1.728462,-1.625550,1.808100,-1.463796
4,IMG_6900_jpeg.rf.530c3f64e20a3233c0421cd9aaa27...,OtherBakingNeeds_400-499g_NonHalal_NonHealthy,-0.894451,-1.714362,-2.250457,-2.714736,-0.854746,-2.529328,-2.300878,-2.260149,...,-0.815020,-0.652744,-0.703792,-0.591155,2.497566,-1.888847,-1.875948,-1.900468,1.467841,-1.778838


In [36]:
big_model_pred_col_name_original = main_imgs_results_big_model.columns[2:].tolist()

In [37]:
new_imgs_results_small_model = pd.read_csv("../../small_model/new_imgs_results_small_model.csv")
new_imgs_results_small_model.head()

,filepath,xmin,ymin,xmax,ymax,label,ProductType,Weight,HalalStatus,HealthStatus,...,Potatochips,RiceBrownOthers,RiceWhite,RolledOatsInstantOatmeal,Salt,SoftDrinksOtherReadyToDrink,SoupStock,Sugar,SweetsChocolatesOthers,TeaPowderLeaves
0,2023_10_25_12_0_34_784872.jpg,285,168,567,528,BabyMilkPowder_200-299g_Halal_NonHealthy,BabyMilkPowder,200-299g,Halal,NonHealthy,...,-1.538441,-2.474456,-2.166450,-2.289746,-1.916843,-0.356248,-1.462294,-1.907547,-1.365697,-2.333017
1,2023_10_25_11_50_16_569076.jpg,266,207,506,528,BabyMilkPowder_400-499g_Halal_NonHealthy,BabyMilkPowder,400-499g,Halal,NonHealthy,...,-0.542177,-2.324751,-1.815467,-2.294456,-1.651892,-1.226976,-2.332330,0.209407,-1.838781,-2.299150
2,2023_10_25_11_45_37_343791.jpg,230,80,562,528,BabyMilkPowder_800-899g_Halal_Healthy,BabyMilkPowder,800-899g,Halal,Healthy,...,1.371769,-1.519539,-0.456094,0.085578,-2.225759,-1.688356,-2.762919,-0.552690,-0.507280,-1.503624
3,2023_10_25_12_9_12_144686.jpg,200,140,532,528,BabyMilkPowder_900-999g_NonHalal_NonHealthy,BabyMilkPowder,900-999g,NonHalal,NonHealthy,...,0.002006,-2.333671,-1.224976,-2.133831,-0.818596,-1.456577,-2.075854,0.011306,-1.141832,-1.817970
4,2023_10_25_12_8_11_460440.jpg,218,125,536,528,BabyMilkPowder_800-899g_Halal_NonHealthy,BabyMilkPowder,800-899g,Halal,NonHealthy,...,-0.578252,-2.372956,-2.174664,-1.919113,-2.649999,-0.679314,-1.787639,-0.066358,-0.924837,-2.095980


In [38]:
# Extract column names that start with 'ProductType'
# all_prdtypes_new_imgs = [col for col in new_imgs_results_small_model.columns if col.startswith('ProductType')]
# all_prdtypes_new_imgs = [col.split('_', 1)[1] for col in all_prdtypes_new_imgs]
# all_prdtypes_new_imgs

# Find the index of the specific column
index_of_specific_column = new_imgs_results_small_model.columns.get_loc('pred_prdtype')

# Retrieve all column names after the specific column
columns_after_specific = new_imgs_results_small_model.columns[index_of_specific_column + 1:]

# Convert to list if you need a list format
all_prdtypes_new_imgs = list(columns_after_specific)
# all_prdtypes_new_imgs

In [39]:
# Check if any name from 'extracted_names' is not in 'df' and add it as a new column
new_prdtype = list(set(all_prdtypes_new_imgs) - set(big_model_pred_col_name))
new_prdtype

[]

In [40]:
if len(new_prdtype)==1:
    main_imgs_results_big_model[new_prdtype[0]] = np.random.normal(loc=CONFIGS["MEAN_PRIOR"], scale=np.sqrt(0.1), size=main_imgs_results_big_model.shape[0])  # Initialize new columns

main_imgs_results_big_model.head()  # Display the updated DataFrame for verificatio

,filepath,label,ProductType_BabyMilkPowder,ProductType_Babyfood,ProductType_BeehoonVermicelli,ProductType_BiscuitsCrackersCookies,ProductType_BreakfastCereals,ProductType_CannedBakedBeans,ProductType_CannedBeefOtherMeats,ProductType_CannedBraisedPeanuts,...,HalalStatus_199g,HalalStatus_299g,HalalStatus_399g,HalalStatus_Halal,HalalStatus_NonHalal,HealthStatus_Halal,HealthStatus_Healthy,HealthStatus_NonHalal,HealthStatus_NonHealthy,HealthStatus_Nonhealthy
0,IMG_2210_JPG.rf.daa853cfd9796f29b5468b3286e774...,BeehoonVermicelli_100-199g_NonHalal_NonHealthy,-2.053605,-1.263907,6.171493,-2.358087,-0.580795,-2.322378,-2.185837,-2.343666,...,-0.649461,-0.664399,-0.755979,-1.253223,3.437621,-1.617361,-1.657442,-1.930155,1.791324,-1.780641
1,output_frame_0591.png,SweetsChocolatesOthers_1-99g_NonHalal_NonHealthy,-1.727152,-1.615095,-1.283436,-1.931547,-1.844568,-1.934808,-1.677901,-1.697859,...,-0.681299,-0.766076,-0.764220,-0.757968,2.670682,-1.326687,-1.483171,-1.412153,1.950356,-1.376114
2,2023_8_11_11_39_48_208811_png.rf.d530c9e0d249e...,BabyMilkPowder_700-799g_Halal_Healthy,7.335651,-3.413930,-3.586535,-3.472556,-2.213640,-3.429164,-2.768110,-2.933477,...,-0.878268,-0.814173,-1.001764,2.501420,-1.337243,-1.802233,2.096767,-1.939219,-2.296831,-1.919529
3,20240219_12_2403.jpg,OtherSauceDressing_200-299g_NonHalal_NonHealthy,-1.844267,-1.954536,-1.982131,-1.682818,-2.304868,-1.458207,-1.998232,-1.795610,...,-0.667080,-0.813992,-0.664773,-0.554538,2.614168,-1.622760,-1.728462,-1.625550,1.808100,-1.463796
4,IMG_6900_jpeg.rf.530c3f64e20a3233c0421cd9aaa27...,OtherBakingNeeds_400-499g_NonHalal_NonHealthy,-0.894451,-1.714362,-2.250457,-2.714736,-0.854746,-2.529328,-2.300878,-2.260149,...,-0.815020,-0.652744,-0.703792,-0.591155,2.497566,-1.888847,-1.875948,-1.900468,1.467841,-1.778838


## Scoring on new imgs

In [41]:
# new_imgs_df = pd.read_csv("../small_model/new_imgs_list.csv")
master_list_df = pd.read_csv(os.path.join('../../', 'master_list.csv'))
new_imgs_df = master_list_df[master_list_df['tag'] == 'TestforMode5']

new_imgs_df.reset_index(drop=True, inplace=True)
new_imgs_df.head()

,filepath,xmin,ymin,xmax,ymax,label,ProductType,Weight,HalalStatus,HealthStatus,new_camera,tag,remove
0,20240209_8_0126.jpg,869,18,1170,694,BeehoonVermicelli_0-99g_Halal_NonHealthy,BeehoonVermicelli,0-99g,Halal,NonHealthy,1,TestforMode5,NaN
1,20240209_8_0137.jpg,707,9,1554,868,BeehoonVermicelli_0-99g_Halal_NonHealthy,BeehoonVermicelli,0-99g,Halal,NonHealthy,1,TestforMode5,NaN
2,20240209_8_0138.jpg,731,10,1560,922,BeehoonVermicelli_0-99g_Halal_NonHealthy,BeehoonVermicelli,0-99g,Halal,NonHealthy,1,TestforMode5,NaN
3,20240209_8_0148.jpg,854,6,1404,900,BeehoonVermicelli_0-99g_Halal_NonHealthy,BeehoonVermicelli,0-99g,Halal,NonHealthy,1,TestforMode5,NaN
4,20240209_8_0150.jpg,809,5,1433,909,BeehoonVermicelli_0-99g_Halal_NonHealthy,BeehoonVermicelli,0-99g,Halal,NonHealthy,1,TestforMode5,NaN


In [42]:
transforms_test = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=CONFIGS['IMG_MEAN'], std=CONFIGS['IMG_STD'])
])

In [60]:
new_imgs_results_big_model = []  # List to store the results
all_prdlabel_preds = []

for idx, row in new_imgs_df.iterrows():
    # image_path = "../small_model/new_imgs/" + row['filepath']
    image_path = os.path.join(CONFIGS["DATA_BASE_PATH"], row['filepath'])
    frame = cv2.imread(image_path)

    # Preprocessing steps
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (CONFIGS['BIG_MODEL_IMG_SIZE'], CONFIGS['BIG_MODEL_IMG_SIZE']))
    frame = frame.transpose((2, 0, 1))
    frame = torch.from_numpy(frame).float()
    frame = transforms_test(frame).unsqueeze(0).to(CONFIGS['DEVICE'])

    # Perform prediction
    with torch.no_grad():
        out1, out2, out3, out4, _ = big_model(frame)

    # Extract and store the results
    prediction_row = [row['filepath'], row['label']]
    prediction_row.extend(out1[0].cpu().numpy().flatten())
    prediction_row.extend(out2[0].cpu().numpy().flatten())
    prediction_row.extend(out3[0].cpu().numpy().flatten())
    prediction_row.extend(out4[0].cpu().numpy().flatten())
    new_imgs_results_big_model.append(prediction_row)

    tmp_pred = out1.argmax(1)
    tmp_pred_prdtype = le_prdtype.inverse_transform([tmp_pred])
    all_prdlabel_preds.append(tmp_pred_prdtype[0])


# Define column names for the new DataFrame
column_names = ['filepath', 'label']
# column_names += big_model_pred_col_name
column_names += ['ProductType_' + name for name in le_prdtype.classes_]
column_names += ['Weight_' + name for name in le_weight.classes_]
column_names += ['HalalStatus_' + name for name in le_halal.classes_]
column_names += ['HealthStatus_' + name for name in le_healthy.classes_]

# Create the DataFrame
new_imgs_results_big_model = pd.DataFrame(new_imgs_results_big_model, columns=column_names)
new_imgs_results_big_model.head()

,filepath,label,ProductType_BabyMilkPowder,ProductType_Babyfood,ProductType_BeehoonVermicelli,ProductType_BiscuitsCrackersCookies,ProductType_BreakfastCereals,ProductType_CannedBakedBeans,ProductType_CannedBeefOtherMeats,ProductType_CannedBraisedPeanuts,...,HalalStatus_199g,HalalStatus_299g,HalalStatus_399g,HalalStatus_Halal,HalalStatus_NonHalal,HealthStatus_Halal,HealthStatus_Healthy,HealthStatus_NonHalal,HealthStatus_NonHealthy,HealthStatus_Nonhealthy
0,20240209_8_0126.jpg,BeehoonVermicelli_0-99g_Halal_NonHealthy,-2.255411,-2.381942,2.772899,-1.176926,-2.251796,-2.141629,-1.877800,-2.037361,...,-0.757450,-0.804775,-0.720020,0.898468,1.157791,-1.622676,-1.431142,-1.716020,1.009496,-0.745693
1,20240209_8_0137.jpg,BeehoonVermicelli_0-99g_Halal_NonHealthy,-1.727243,-2.163061,4.672274,-1.832929,-1.825044,-1.829648,-1.953308,-1.901429,...,-0.817711,-0.654699,-0.670969,1.051013,1.151079,-1.760566,-1.359725,-1.652903,1.024580,-1.205348
2,20240209_8_0138.jpg,BeehoonVermicelli_0-99g_Halal_NonHealthy,-1.737916,-2.008745,5.147163,-2.329475,-2.016395,-1.988709,-1.923268,-2.210156,...,-0.850687,-0.629788,-0.721646,1.249366,0.934755,-1.795048,-1.697518,-1.675058,1.394100,-1.464122
3,20240209_8_0148.jpg,BeehoonVermicelli_0-99g_Halal_NonHealthy,-1.787705,-2.090640,4.986648,-2.483324,-2.050333,-1.993925,-1.843712,-2.037488,...,-0.782138,-0.692990,-0.682241,1.329572,0.823488,-1.588717,-1.453506,-1.541076,1.032548,-0.908719
4,20240209_8_0150.jpg,BeehoonVermicelli_0-99g_Halal_NonHealthy,-1.903943,-1.863413,5.574066,-2.492425,-1.862761,-1.958135,-1.837561,-2.156187,...,-0.798959,-0.682592,-0.706129,1.296911,0.832826,-1.590535,-1.542661,-1.608659,1.380229,-1.175634


In [61]:
print("Accuracy on new images")
print(sum(all_prdlabel_preds == new_imgs_df['ProductType']) / len(new_imgs_df))

Accuracy on new images
1.0


In [64]:
all_prdlabel_preds

['BeehoonVermicelli',
 'BeehoonVermicelli',
 'BeehoonVermicelli',
 'BeehoonVermicelli',
 'BeehoonVermicelli',
 'BeehoonVermicelli',
 'BeehoonVermicelli',
 'BeehoonVermicelli',
 'BeehoonVermicelli',
 'BeehoonVermicelli']

In [45]:
new_imgs_results_big_model['pred_prdtype'] = new_imgs_results_big_model[big_model_pred_col_name].idxmax(axis=1)
target_column = 'pred_prdtype'
columns = [target_column] + [col for col in new_imgs_results_big_model.columns if col != target_column]
new_imgs_results_big_model = new_imgs_results_big_model[columns]
new_imgs_results_big_model.head()

,pred_prdtype,filepath,label,BabyMilkPowder,Babyfood,BeehoonVermicelli,BiscuitsCrackersCookies,BreakfastCereals,CannedBakedBeans,CannedBeefOtherMeats,...,Potatochips,RiceBrownOthers,RiceWhite,RolledOatsInstantOatmeal,Salt,SoftDrinksOtherReadyToDrink,SoupStock,Sugar,SweetsChocolatesOthers,TeaPowderLeaves
0,BeehoonVermicelli,20240209_8_0126.jpg,BeehoonVermicelli_0-99g_Halal_NonHealthy,-2.255411,-2.381942,2.772899,-1.176926,-2.251796,-2.141629,-1.877800,...,-2.471951,-1.583861,-2.160729,-1.358500,-2.252579,-2.034832,-1.808990,-2.036037,1.238598,-2.237613
1,BeehoonVermicelli,20240209_8_0137.jpg,BeehoonVermicelli_0-99g_Halal_NonHealthy,-1.727243,-2.163061,4.672274,-1.832929,-1.825044,-1.829648,-1.953308,...,-0.862594,-1.681263,-2.529849,-1.354852,-2.270963,-1.968993,-1.766751,-2.209167,-0.885389,-1.773730
2,BeehoonVermicelli,20240209_8_0138.jpg,BeehoonVermicelli_0-99g_Halal_NonHealthy,-1.737916,-2.008745,5.147163,-2.329475,-2.016395,-1.988709,-1.923268,...,-0.001525,-2.082963,-2.619079,-1.910997,-2.488396,-1.880246,-1.724994,-2.411345,-1.448188,-1.928989
3,BeehoonVermicelli,20240209_8_0148.jpg,BeehoonVermicelli_0-99g_Halal_NonHealthy,-1.787705,-2.090640,4.986648,-2.483324,-2.050333,-1.993925,-1.843712,...,-2.074385,-1.688260,-2.315485,-1.647893,-2.341962,-1.832736,-1.644695,-2.138435,-0.059624,-2.039414
4,BeehoonVermicelli,20240209_8_0150.jpg,BeehoonVermicelli_0-99g_Halal_NonHealthy,-1.903943,-1.863413,5.574066,-2.492425,-1.862761,-1.958135,-1.837561,...,-2.161538,-1.930350,-2.273765,-1.855097,-2.370565,-1.786512,-1.662063,-2.209034,-0.582619,-1.927492


In [46]:
if len(new_prdtype)==1:
    new_imgs_results_big_model[new_prdtype[0]] = np.random.normal(loc=CONFIGS["MEAN_PRIOR"], scale=np.sqrt(0.1), size=new_imgs_results_big_model.shape[0])  # Initialize new columns

new_imgs_results_big_model.head()  # Display the updated DataFrame for verificatio

,pred_prdtype,filepath,label,BabyMilkPowder,Babyfood,BeehoonVermicelli,BiscuitsCrackersCookies,BreakfastCereals,CannedBakedBeans,CannedBeefOtherMeats,...,Potatochips,RiceBrownOthers,RiceWhite,RolledOatsInstantOatmeal,Salt,SoftDrinksOtherReadyToDrink,SoupStock,Sugar,SweetsChocolatesOthers,TeaPowderLeaves
0,BeehoonVermicelli,20240209_8_0126.jpg,BeehoonVermicelli_0-99g_Halal_NonHealthy,-2.255411,-2.381942,2.772899,-1.176926,-2.251796,-2.141629,-1.877800,...,-2.471951,-1.583861,-2.160729,-1.358500,-2.252579,-2.034832,-1.808990,-2.036037,1.238598,-2.237613
1,BeehoonVermicelli,20240209_8_0137.jpg,BeehoonVermicelli_0-99g_Halal_NonHealthy,-1.727243,-2.163061,4.672274,-1.832929,-1.825044,-1.829648,-1.953308,...,-0.862594,-1.681263,-2.529849,-1.354852,-2.270963,-1.968993,-1.766751,-2.209167,-0.885389,-1.773730
2,BeehoonVermicelli,20240209_8_0138.jpg,BeehoonVermicelli_0-99g_Halal_NonHealthy,-1.737916,-2.008745,5.147163,-2.329475,-2.016395,-1.988709,-1.923268,...,-0.001525,-2.082963,-2.619079,-1.910997,-2.488396,-1.880246,-1.724994,-2.411345,-1.448188,-1.928989
3,BeehoonVermicelli,20240209_8_0148.jpg,BeehoonVermicelli_0-99g_Halal_NonHealthy,-1.787705,-2.090640,4.986648,-2.483324,-2.050333,-1.993925,-1.843712,...,-2.074385,-1.688260,-2.315485,-1.647893,-2.341962,-1.832736,-1.644695,-2.138435,-0.059624,-2.039414
4,BeehoonVermicelli,20240209_8_0150.jpg,BeehoonVermicelli_0-99g_Halal_NonHealthy,-1.903943,-1.863413,5.574066,-2.492425,-1.862761,-1.958135,-1.837561,...,-2.161538,-1.930350,-2.273765,-1.855097,-2.370565,-1.786512,-1.662063,-2.209034,-0.582619,-1.927492


In [47]:
new_imgs_results_big_model.shape

(10, 70)

In [48]:
main_imgs_results_big_model.shape

(6438, 97)

## All scorings from big model

In [49]:
main_imgs_results_big_model.head()

,filepath,label,ProductType_BabyMilkPowder,ProductType_Babyfood,ProductType_BeehoonVermicelli,ProductType_BiscuitsCrackersCookies,ProductType_BreakfastCereals,ProductType_CannedBakedBeans,ProductType_CannedBeefOtherMeats,ProductType_CannedBraisedPeanuts,...,HalalStatus_199g,HalalStatus_299g,HalalStatus_399g,HalalStatus_Halal,HalalStatus_NonHalal,HealthStatus_Halal,HealthStatus_Healthy,HealthStatus_NonHalal,HealthStatus_NonHealthy,HealthStatus_Nonhealthy
0,IMG_2210_JPG.rf.daa853cfd9796f29b5468b3286e774...,BeehoonVermicelli_100-199g_NonHalal_NonHealthy,-2.053605,-1.263907,6.171493,-2.358087,-0.580795,-2.322378,-2.185837,-2.343666,...,-0.649461,-0.664399,-0.755979,-1.253223,3.437621,-1.617361,-1.657442,-1.930155,1.791324,-1.780641
1,output_frame_0591.png,SweetsChocolatesOthers_1-99g_NonHalal_NonHealthy,-1.727152,-1.615095,-1.283436,-1.931547,-1.844568,-1.934808,-1.677901,-1.697859,...,-0.681299,-0.766076,-0.764220,-0.757968,2.670682,-1.326687,-1.483171,-1.412153,1.950356,-1.376114
2,2023_8_11_11_39_48_208811_png.rf.d530c9e0d249e...,BabyMilkPowder_700-799g_Halal_Healthy,7.335651,-3.413930,-3.586535,-3.472556,-2.213640,-3.429164,-2.768110,-2.933477,...,-0.878268,-0.814173,-1.001764,2.501420,-1.337243,-1.802233,2.096767,-1.939219,-2.296831,-1.919529
3,20240219_12_2403.jpg,OtherSauceDressing_200-299g_NonHalal_NonHealthy,-1.844267,-1.954536,-1.982131,-1.682818,-2.304868,-1.458207,-1.998232,-1.795610,...,-0.667080,-0.813992,-0.664773,-0.554538,2.614168,-1.622760,-1.728462,-1.625550,1.808100,-1.463796
4,IMG_6900_jpeg.rf.530c3f64e20a3233c0421cd9aaa27...,OtherBakingNeeds_400-499g_NonHalal_NonHealthy,-0.894451,-1.714362,-2.250457,-2.714736,-0.854746,-2.529328,-2.300878,-2.260149,...,-0.815020,-0.652744,-0.703792,-0.591155,2.497566,-1.888847,-1.875948,-1.900468,1.467841,-1.778838


In [50]:
main_imgs_results_big_model['type'] = "existing"
new_imgs_results_big_model['type'] = "new"

new_columns = [col.replace('ProductType_', '', 1) if col.startswith('ProductType_') else col for col in main_imgs_results_big_model.columns]
main_imgs_results_big_model.columns = new_columns

all_imgs_results_big_model = pd.concat([main_imgs_results_big_model, new_imgs_results_big_model], axis=0)
all_imgs_results_big_model.reset_index(drop=True, inplace=True)
all_imgs_results_big_model.head()

,filepath,label,BabyMilkPowder,Babyfood,BeehoonVermicelli,BiscuitsCrackersCookies,BreakfastCereals,CannedBakedBeans,CannedBeefOtherMeats,CannedBraisedPeanuts,...,HalalStatus_399g,HalalStatus_Halal,HalalStatus_NonHalal,HealthStatus_Halal,HealthStatus_Healthy,HealthStatus_NonHalal,HealthStatus_NonHealthy,HealthStatus_Nonhealthy,type,pred_prdtype
0,IMG_2210_JPG.rf.daa853cfd9796f29b5468b3286e774...,BeehoonVermicelli_100-199g_NonHalal_NonHealthy,-2.053605,-1.263907,6.171493,-2.358087,-0.580795,-2.322378,-2.185837,-2.343666,...,-0.755979,-1.253223,3.437621,-1.617361,-1.657442,-1.930155,1.791324,-1.780641,existing,NaN
1,output_frame_0591.png,SweetsChocolatesOthers_1-99g_NonHalal_NonHealthy,-1.727152,-1.615095,-1.283436,-1.931547,-1.844568,-1.934808,-1.677901,-1.697859,...,-0.764220,-0.757968,2.670682,-1.326687,-1.483171,-1.412153,1.950356,-1.376114,existing,NaN
2,2023_8_11_11_39_48_208811_png.rf.d530c9e0d249e...,BabyMilkPowder_700-799g_Halal_Healthy,7.335651,-3.413930,-3.586535,-3.472556,-2.213640,-3.429164,-2.768110,-2.933477,...,-1.001764,2.501420,-1.337243,-1.802233,2.096767,-1.939219,-2.296831,-1.919529,existing,NaN
3,20240219_12_2403.jpg,OtherSauceDressing_200-299g_NonHalal_NonHealthy,-1.844267,-1.954536,-1.982131,-1.682818,-2.304868,-1.458207,-1.998232,-1.795610,...,-0.664773,-0.554538,2.614168,-1.622760,-1.728462,-1.625550,1.808100,-1.463796,existing,NaN
4,IMG_6900_jpeg.rf.530c3f64e20a3233c0421cd9aaa27...,OtherBakingNeeds_400-499g_NonHalal_NonHealthy,-0.894451,-1.714362,-2.250457,-2.714736,-0.854746,-2.529328,-2.300878,-2.260149,...,-0.703792,-0.591155,2.497566,-1.888847,-1.875948,-1.900468,1.467841,-1.778838,existing,NaN


In [51]:
all_imgs_results_big_model.tail()

,filepath,label,BabyMilkPowder,Babyfood,BeehoonVermicelli,BiscuitsCrackersCookies,BreakfastCereals,CannedBakedBeans,CannedBeefOtherMeats,CannedBraisedPeanuts,...,HalalStatus_399g,HalalStatus_Halal,HalalStatus_NonHalal,HealthStatus_Halal,HealthStatus_Healthy,HealthStatus_NonHalal,HealthStatus_NonHealthy,HealthStatus_Nonhealthy,type,pred_prdtype
6443,20240209_8_0158.jpg,BeehoonVermicelli_0-99g_Halal_NonHealthy,-1.557630,-2.111372,1.877454,-1.020206,-1.864454,-1.367910,-1.305434,-1.440210,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,new,BeehoonVermicelli
6444,20240209_8_0159.jpg,BeehoonVermicelli_0-99g_Halal_NonHealthy,-1.496348,-1.867517,1.950424,-0.545495,-1.681382,-1.458135,-1.406064,-1.459004,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,new,BeehoonVermicelli
6445,20240209_8_0168.jpg,BeehoonVermicelli_0-99g_Halal_NonHealthy,-1.772027,-2.525744,3.145372,-0.452234,-1.975837,-2.024747,-2.090684,-1.954216,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,new,BeehoonVermicelli
6446,20240209_8_0179.jpg,BeehoonVermicelli_0-99g_Halal_NonHealthy,-1.810689,-2.073942,4.131886,-2.596778,-1.460403,-1.893325,-1.814744,-2.059152,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,new,BeehoonVermicelli
6447,20240209_8_0180.jpg,BeehoonVermicelli_0-99g_Halal_NonHealthy,-1.749967,-2.179266,4.065598,-2.404998,-1.478165,-1.863546,-1.755984,-1.994669,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,new,BeehoonVermicelli


In [52]:
all_imgs_results_big_model.shape

(6448, 99)

In [53]:
all_imgs_results_big_model.to_csv("output/all_imgs_results_big_model.csv", index=True)

In [26]:
main_imgs_results_big_model.head()

,filepath,label,BabyMilkPowder,Babyfood,BeehoonVermicelli,BiscuitsCrackersCookies,BreakfastCereals,CannedBakedBeans,CannedBeefOtherMeats,CannedBraisedPeanuts,...,Weight_600-699g,Weight_700-799g,Weight_800-899g,Weight_900-999g,HalalStatus_Halal,HalalStatus_NonHalal,HealthStatus_Healthy,HealthStatus_NonHealthy,Coffee,type
0,IMG_5409_jpeg.rf.4720409117e10800de5acb5412b3b...,OtherSauceDressing_1-99g_NonHalal_NonHealthy,-2.331182,-1.358775,-2.555626,-2.238294,-2.599167,-1.646559,-2.415531,-1.704499,...,-0.930527,-1.185138,-1.278101,-1.013106,-0.956202,0.636404,-0.617663,0.979402,-15.676090,existing
1,output_frame_0527.png,Potatochips_1-99g_NonHalal_NonHealthy,-1.560478,-1.479097,-1.757943,-1.280493,-1.098393,-1.589285,-1.746585,-1.463236,...,-0.877150,-0.815971,-0.809895,-0.808688,-1.149344,0.515621,-0.741172,0.900027,-15.072512,existing
2,output_frame_0415.png,Nuts_100-199g_Halal_NonHealthy,-1.622234,-1.469053,-2.129289,-1.152001,-1.575088,-1.785793,-1.523244,-1.593447,...,-0.811816,-0.857038,-0.784750,-0.829949,0.794045,-1.182505,-0.781884,0.853192,-14.729252,existing
3,IMG_5379_jpeg.rf.1e3e5a76729c48bcf7e6b3624865a...,BiscuitsCrackersCookies_100-199g_Halal_NonHealthy,-1.530578,-2.011827,-1.517158,5.906901,-2.058439,-2.133619,-2.141947,-1.930500,...,-1.016780,-0.892647,-1.169003,-0.831979,0.460972,-1.137421,-0.698091,0.872436,-15.160820,existing
4,output_frame_0081.png,RolledOatsInstantOatmeal_400-499g_Halal_Healthy,-2.057776,-1.719505,-1.694581,-1.306111,-1.572085,-1.655526,-2.094795,-1.643083,...,-0.595777,-0.940049,-0.861996,-1.112873,0.958680,-0.844323,0.916633,-0.951688,-14.441660,existing


# Small model (not used)

## Model loading

In [21]:
class MultiHeadResNet_SmallModel(nn.Module):
    def __init__(self, num_classes_prdtype, num_classes_weight, num_classes_halal, num_classes_healthy):
        super(MultiHeadResNet_SmallModel, self).__init__()
        self.base_model = models.resnet18(pretrained=True)
        num_ftrs = self.base_model.fc.in_features
        self.base_model.fc = nn.Identity()

        # Define custom fully connected layers for each prediction head
        self.fc_prdtype = nn.Linear(num_ftrs, num_classes_prdtype)
        self.fc_weight = nn.Linear(num_ftrs, num_classes_weight)
        self.fc_halal = nn.Linear(num_ftrs, num_classes_halal)
        self.fc_healthy = nn.Linear(num_ftrs, num_classes_healthy)

    def forward(self, x):
        x = self.base_model(x)
        prdtype = self.fc_prdtype(x)
        weight = self.fc_weight(x)
        halal = self.fc_halal(x)
        healthy = self.fc_healthy(x)
        return prdtype, weight, halal, healthy

    
# load label encoder 
def load_label_encoder_small_model():
    le_prdtype = pickle.loads(open("../small_model/output/le_prdtype.pickle", "rb").read())
    le_weight = pickle.loads(open("../small_model/output/le_weight.pickle", "rb").read())
    le_halal = pickle.loads(open("../small_model/output/le_halal.pickle", "rb").read())
    le_healthy = pickle.loads(open("../small_model/output/le_healthy.pickle", "rb").read())
    
    return le_prdtype, le_weight, le_halal, le_healthy

le_prdtype, le_weight, le_halal, le_healthy = load_label_encoder_small_model()

# Load the trained MultiHeadResNet model
def load_model():
    # Verify the number of classes for each label
    num_classes_prdtype = len(le_prdtype.classes_)
    num_classes_weight = len(le_weight.classes_)
    num_classes_halal = len(le_halal.classes_)
    num_classes_healthy = len(le_healthy.classes_)
    # print(num_classes_prdtype)
    # print(num_classes_healthy)

    custom_resnet_model = MultiHeadResNet_SmallModel(
        num_classes_prdtype=num_classes_prdtype,
        num_classes_weight=num_classes_weight,
        num_classes_halal=num_classes_halal,
        num_classes_healthy=num_classes_healthy
    )

    model_path = '../small_model/output/multi_head_model.pth'
    # print("test1")
    if os.path.exists(model_path):
        custom_resnet_model.load_state_dict(torch.load(model_path, map_location=CONFIGS['DEVICE']))
    else:
        raise FileNotFoundError(f"Model file not found: {model_path}")
    # print("test2")
    custom_resnet_model.to(CONFIGS['DEVICE'])
    custom_resnet_model.eval()
    return custom_resnet_model
 
small_model = load_model()

/Users/liupeng/opt/anaconda3/envs/py37/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/Users/liupeng/opt/anaconda3/envs/py37/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Scoring on new imgs 

In [22]:
new_imgs_df = pd.read_csv("../small_model/new_imgs_list.csv")
new_imgs_df.reset_index(drop=True, inplace=True)

# ADHOC: change the new imgs to existing type
new_imgs_df['label'] = 'AdultMilk_1-99g_Halal_NonHealthy'
new_imgs_df['ProductType'] = 'AdultMilk'
new_imgs_df['Weight'] = '1-99g'
new_imgs_df['HalalStatus'] = 'Halal'
new_imgs_df['HealthStatus'] = 'NonHealthy'

new_imgs_df.head()

,filepath,label,ProductType,Weight,HalalStatus,HealthStatus
0,5131704785418_.pic.jpg,AdultMilk_1-99g_Halal_NonHealthy,AdultMilk,1-99g,Halal,NonHealthy
1,5141704785419_.pic.jpg,AdultMilk_1-99g_Halal_NonHealthy,AdultMilk,1-99g,Halal,NonHealthy
2,5151704785420_.pic.jpg,AdultMilk_1-99g_Halal_NonHealthy,AdultMilk,1-99g,Halal,NonHealthy
3,5161704785422_.pic.jpg,AdultMilk_1-99g_Halal_NonHealthy,AdultMilk,1-99g,Halal,NonHealthy
4,5171704785423_.pic.jpg,AdultMilk_1-99g_Halal_NonHealthy,AdultMilk,1-99g,Halal,NonHealthy


In [23]:
new_imgs_results_small_model = pd.read_csv("../small_model/new_imgs_results_small_model.csv")
new_imgs_results_small_model = new_imgs_results_small_model.loc[new_imgs_results_small_model.Filename.isin(new_imgs_df.filepath)]
new_imgs_results_small_model.reset_index(drop=True, inplace=True)
new_imgs_results_small_model.head()

,Filename,CorrectTotalLabel,ProductType_AdultMilk,ProductType_BabyMilkPowder,ProductType_Babyfood,ProductType_BeehoonVermicelliMeesua,ProductType_BiscuitsCrackersCookies,ProductType_Book,ProductType_BreakfastCerealsCornflakes,ProductType_CannedPacketCreamersSweet,...,Weight_400-499g,Weight_500-599g,Weight_600-699g,Weight_700-799g,Weight_800-899g,Weight_900-999g,HalalStatus_Halal,HalalStatus_NonHalal,HealthStatus_Healthy,HealthStatus_NonHealthy
0,5191704785428_.pic.jpg,AdultMilk_1-99g_Halal_NonHealthy,7.467985,-0.455903,-1.165236,-0.801069,-1.308381,-3.237241,-4.273062,-2.624339,...,-0.989483,-1.511478,-1.901927,-1.902937,-1.082075,-1.482124,2.499995,-3.375038,-3.133447,1.911802
1,5131704785418_.pic.jpg,AdultMilk_1-99g_Halal_NonHealthy,10.115109,-0.741757,-1.852127,-2.395665,-1.157235,-3.507548,-4.208895,-1.817594,...,-1.521327,-3.257041,-1.830936,-2.256523,-1.258991,-1.090913,3.655709,-5.586953,-4.097594,3.037369
2,5161704785422_.pic.jpg,AdultMilk_1-99g_Halal_NonHealthy,10.826588,-0.742384,-0.055338,-3.254927,0.313838,-3.043004,-3.574817,-1.882671,...,0.850131,-3.959396,-0.241461,-2.913476,-0.968537,-1.842475,4.468235,-5.253281,-4.024357,4.338975
3,5221704785433_.pic.jpg,AdultMilk_1-99g_Halal_NonHealthy,10.402787,-1.199843,-1.399939,-3.267868,-0.732076,-2.685109,-4.122629,-1.684803,...,-1.405638,-3.556877,-0.852787,-1.800704,-0.837352,-1.654783,2.876932,-4.567831,-4.165457,3.584487
4,5171704785423_.pic.jpg,AdultMilk_1-99g_Halal_NonHealthy,9.232785,-0.722910,-1.045706,-1.955620,-0.930536,-2.480214,-3.910032,-1.965349,...,-1.367624,-3.448782,-0.539122,-1.300058,-0.615255,-1.874075,2.897145,-3.574468,-4.037078,3.543715


In [24]:
new_imgs_results_small_model.shape

(10, 59)

In [25]:
# Create a copy of the current column names to a list
new_columns = new_imgs_results_small_model.columns.tolist()

# Modify the first two elements
new_columns[0] = 'filepath'
new_columns[1] = 'label'

# Assign the modified list of column names back to the DataFrame
new_imgs_results_small_model.columns = new_columns
new_imgs_results_small_model.head()

,filepath,label,ProductType_AdultMilk,ProductType_BabyMilkPowder,ProductType_Babyfood,ProductType_BeehoonVermicelliMeesua,ProductType_BiscuitsCrackersCookies,ProductType_Book,ProductType_BreakfastCerealsCornflakes,ProductType_CannedPacketCreamersSweet,...,Weight_400-499g,Weight_500-599g,Weight_600-699g,Weight_700-799g,Weight_800-899g,Weight_900-999g,HalalStatus_Halal,HalalStatus_NonHalal,HealthStatus_Healthy,HealthStatus_NonHealthy
0,5191704785428_.pic.jpg,AdultMilk_1-99g_Halal_NonHealthy,7.467985,-0.455903,-1.165236,-0.801069,-1.308381,-3.237241,-4.273062,-2.624339,...,-0.989483,-1.511478,-1.901927,-1.902937,-1.082075,-1.482124,2.499995,-3.375038,-3.133447,1.911802
1,5131704785418_.pic.jpg,AdultMilk_1-99g_Halal_NonHealthy,10.115109,-0.741757,-1.852127,-2.395665,-1.157235,-3.507548,-4.208895,-1.817594,...,-1.521327,-3.257041,-1.830936,-2.256523,-1.258991,-1.090913,3.655709,-5.586953,-4.097594,3.037369
2,5161704785422_.pic.jpg,AdultMilk_1-99g_Halal_NonHealthy,10.826588,-0.742384,-0.055338,-3.254927,0.313838,-3.043004,-3.574817,-1.882671,...,0.850131,-3.959396,-0.241461,-2.913476,-0.968537,-1.842475,4.468235,-5.253281,-4.024357,4.338975
3,5221704785433_.pic.jpg,AdultMilk_1-99g_Halal_NonHealthy,10.402787,-1.199843,-1.399939,-3.267868,-0.732076,-2.685109,-4.122629,-1.684803,...,-1.405638,-3.556877,-0.852787,-1.800704,-0.837352,-1.654783,2.876932,-4.567831,-4.165457,3.584487
4,5171704785423_.pic.jpg,AdultMilk_1-99g_Halal_NonHealthy,9.232785,-0.722910,-1.045706,-1.955620,-0.930536,-2.480214,-3.910032,-1.965349,...,-1.367624,-3.448782,-0.539122,-1.300058,-0.615255,-1.874075,2.897145,-3.574468,-4.037078,3.543715


In [26]:
# Check if any name from 'extracted_names' is not in 'df' and add it as a new column
new_prdtype = list(set(all_imgs_results_big_model.columns) - set(new_imgs_results_small_model.columns))

if len(new_prdtype)>0:
    for col in new_prdtype:
        new_imgs_results_small_model[col] = np.random.normal(loc=CONFIGS["MEAN_PRIOR"], scale=np.sqrt(0.1), size=new_imgs_results_small_model.shape[0])  # Initialize new columns

new_imgs_results_small_model.head()  # Display the updated DataFrame for verificatio

,filepath,label,ProductType_AdultMilk,ProductType_BabyMilkPowder,ProductType_Babyfood,ProductType_BeehoonVermicelliMeesua,ProductType_BiscuitsCrackersCookies,ProductType_Book,ProductType_BreakfastCerealsCornflakes,ProductType_CannedPacketCreamersSweet,...,Weight_500-599g,Weight_600-699g,Weight_700-799g,Weight_800-899g,Weight_900-999g,HalalStatus_Halal,HalalStatus_NonHalal,HealthStatus_Healthy,HealthStatus_NonHealthy,img_type
0,5191704785428_.pic.jpg,AdultMilk_1-99g_Halal_NonHealthy,7.467985,-0.455903,-1.165236,-0.801069,-1.308381,-3.237241,-4.273062,-2.624339,...,-1.511478,-1.901927,-1.902937,-1.082075,-1.482124,2.499995,-3.375038,-3.133447,1.911802,-14.907014
1,5131704785418_.pic.jpg,AdultMilk_1-99g_Halal_NonHealthy,10.115109,-0.741757,-1.852127,-2.395665,-1.157235,-3.507548,-4.208895,-1.817594,...,-3.257041,-1.830936,-2.256523,-1.258991,-1.090913,3.655709,-5.586953,-4.097594,3.037369,-15.152727
2,5161704785422_.pic.jpg,AdultMilk_1-99g_Halal_NonHealthy,10.826588,-0.742384,-0.055338,-3.254927,0.313838,-3.043004,-3.574817,-1.882671,...,-3.959396,-0.241461,-2.913476,-0.968537,-1.842475,4.468235,-5.253281,-4.024357,4.338975,-14.836858
3,5221704785433_.pic.jpg,AdultMilk_1-99g_Halal_NonHealthy,10.402787,-1.199843,-1.399939,-3.267868,-0.732076,-2.685109,-4.122629,-1.684803,...,-3.556877,-0.852787,-1.800704,-0.837352,-1.654783,2.876932,-4.567831,-4.165457,3.584487,-15.454279
4,5171704785423_.pic.jpg,AdultMilk_1-99g_Halal_NonHealthy,9.232785,-0.722910,-1.045706,-1.955620,-0.930536,-2.480214,-3.910032,-1.965349,...,-3.448782,-0.539122,-1.300058,-0.615255,-1.874075,2.897145,-3.574468,-4.037078,3.543715,-15.328803


In [27]:
new_imgs_results_small_model.shape

(10, 60)

## Scoring on main imgs

In [28]:
main_imgs_master_list = pd.read_csv("../master_list.csv")
main_imgs_master_list.head()

,filepath,xmin,ymin,xmax,ymax,label,ProductType,Weight,HalalStatus,HealthStatus,new_camera,tag
0,IMG_20230428_123528_jpg.rf.5687b7b914f6d9aa98c...,151,42,497,591,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN
1,IMG_20230428_123522_jpg.rf.204ff37f497f2dce442...,88,81,442,567,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN
2,IMG_20230428_123708_jpg.rf.141ecd0cefaea75c0b7...,35,34,492,622,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN
3,IMG_20230428_123521_jpg.rf.1069b402272252862ec...,99,122,428,587,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN
4,IMG_20230428_123659_jpg.rf.5e1b6c4caabe48cf360...,103,17,474,592,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN


In [29]:
main_imgs_results_small_model = []  # List to store the results
le_prdtype, le_weight, le_halal, le_healthy = load_label_encoder_small_model()

for idx, row in main_imgs_master_list.iterrows():
    image_path = "../all_images/" + row['filepath']
    frame = cv2.imread(image_path)

    # Preprocessing steps
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (CONFIGS['SMALL_MODEL_IMG_SIZE'], CONFIGS['SMALL_MODEL_IMG_SIZE']))
    frame = frame.transpose((2, 0, 1))
    frame = torch.from_numpy(frame).float()
    frame = transforms_test(frame).unsqueeze(0).to(CONFIGS['DEVICE'])

    # Perform prediction
    with torch.no_grad():
        out1, out2, out3, out4 = small_model(frame)
    
    # Extract and store the results
    prediction_row = [row['filepath'], row['label']]
    prediction_row.extend(out1.cpu().numpy().flatten())
    prediction_row.extend(out2.cpu().numpy().flatten())
    prediction_row.extend(out3.cpu().numpy().flatten())
    prediction_row.extend(out4.cpu().numpy().flatten())
    main_imgs_results_small_model.append(prediction_row)


# Define column names for the new DataFrame
column_names = ['filepath', 'label']
column_names += ['ProductType_' + name for name in le_prdtype.classes_]
column_names += ['Weight_' + name for name in le_weight.classes_]
column_names += ['HalalStatus_' + name for name in le_halal.classes_]
column_names += ['HealthStatus_' + name for name in le_healthy.classes_]


# Create the DataFrame
main_imgs_results_small_model = pd.DataFrame(main_imgs_results_small_model, columns=column_names)
main_imgs_results_small_model.head()

,filepath,label,ProductType_AdultMilk,ProductType_BabyMilkPowder,ProductType_Babyfood,ProductType_BeehoonVermicelliMeesua,ProductType_BiscuitsCrackersCookies,ProductType_Book,ProductType_BreakfastCerealsCornflakes,ProductType_CannedPacketCreamersSweet,...,Weight_400-499g,Weight_500-599g,Weight_600-699g,Weight_700-799g,Weight_800-899g,Weight_900-999g,HalalStatus_Halal,HalalStatus_NonHalal,HealthStatus_Healthy,HealthStatus_NonHealthy
0,IMG_20230428_123528_jpg.rf.5687b7b914f6d9aa98c...,Sugar_400-499g_NonHalal_NonHealthy,-3.593526,-0.834642,-0.036741,-0.449187,2.034535,-2.399136,-1.877603,-3.115813,...,7.441017,-1.874432,-1.630215,-3.345615,0.154984,-4.595025,-1.709256,0.742516,-4.266921,1.251087
1,IMG_20230428_123522_jpg.rf.204ff37f497f2dce442...,Sugar_400-499g_NonHalal_NonHealthy,-3.660402,-2.571959,0.224866,-2.183870,-0.816285,-2.118711,-1.726907,-4.019604,...,6.182879,-2.065965,-2.811384,-5.012823,0.327200,-1.995343,-1.523705,1.806258,-4.367521,0.752712
2,IMG_20230428_123708_jpg.rf.141ecd0cefaea75c0b7...,Sugar_400-499g_NonHalal_NonHealthy,-0.960679,-3.024067,0.840389,-1.106379,-1.344823,-1.859851,-0.977400,-2.414862,...,7.435597,0.022743,-2.701884,-5.151578,-0.955235,-1.765768,-3.311442,2.412364,-4.632944,4.555023
3,IMG_20230428_123521_jpg.rf.1069b402272252862ec...,Sugar_400-499g_NonHalal_NonHealthy,-2.445158,-2.502353,-0.030090,-2.150193,-0.496334,-1.863340,-1.602406,-2.944317,...,5.663602,-1.559119,-3.432280,-4.082855,-0.294293,-0.931375,-1.490215,1.054060,-3.768671,-0.287054
4,IMG_20230428_123659_jpg.rf.5e1b6c4caabe48cf360...,Sugar_400-499g_NonHalal_NonHealthy,-3.006482,-2.206970,-2.354793,-1.360836,-0.575183,-2.720716,-0.906097,-2.562416,...,8.325328,-0.317981,-1.517341,-4.780657,-3.586008,-2.380914,-5.034392,3.682207,-6.363514,2.529441


In [30]:
main_imgs_results_small_model.shape

(4458, 59)

In [31]:
# Check if any name from 'extracted_names' is not in 'df' and add it as a new column
new_prdtype = list(set(all_imgs_results_big_model.columns) - set(main_imgs_results_small_model.columns))

if len(new_prdtype)>0:
    for col in new_prdtype:
        main_imgs_results_small_model[col] = np.random.normal(loc=CONFIGS["MEAN_PRIOR"], scale=np.sqrt(0.1), size=main_imgs_results_small_model.shape[0])  # Initialize new columns

main_imgs_results_small_model.head()  # Display the updated DataFrame for verificatio

,filepath,label,ProductType_AdultMilk,ProductType_BabyMilkPowder,ProductType_Babyfood,ProductType_BeehoonVermicelliMeesua,ProductType_BiscuitsCrackersCookies,ProductType_Book,ProductType_BreakfastCerealsCornflakes,ProductType_CannedPacketCreamersSweet,...,Weight_500-599g,Weight_600-699g,Weight_700-799g,Weight_800-899g,Weight_900-999g,HalalStatus_Halal,HalalStatus_NonHalal,HealthStatus_Healthy,HealthStatus_NonHealthy,img_type
0,IMG_20230428_123528_jpg.rf.5687b7b914f6d9aa98c...,Sugar_400-499g_NonHalal_NonHealthy,-3.593526,-0.834642,-0.036741,-0.449187,2.034535,-2.399136,-1.877603,-3.115813,...,-1.874432,-1.630215,-3.345615,0.154984,-4.595025,-1.709256,0.742516,-4.266921,1.251087,-14.744955
1,IMG_20230428_123522_jpg.rf.204ff37f497f2dce442...,Sugar_400-499g_NonHalal_NonHealthy,-3.660402,-2.571959,0.224866,-2.183870,-0.816285,-2.118711,-1.726907,-4.019604,...,-2.065965,-2.811384,-5.012823,0.327200,-1.995343,-1.523705,1.806258,-4.367521,0.752712,-14.986271
2,IMG_20230428_123708_jpg.rf.141ecd0cefaea75c0b7...,Sugar_400-499g_NonHalal_NonHealthy,-0.960679,-3.024067,0.840389,-1.106379,-1.344823,-1.859851,-0.977400,-2.414862,...,0.022743,-2.701884,-5.151578,-0.955235,-1.765768,-3.311442,2.412364,-4.632944,4.555023,-15.342698
3,IMG_20230428_123521_jpg.rf.1069b402272252862ec...,Sugar_400-499g_NonHalal_NonHealthy,-2.445158,-2.502353,-0.030090,-2.150193,-0.496334,-1.863340,-1.602406,-2.944317,...,-1.559119,-3.432280,-4.082855,-0.294293,-0.931375,-1.490215,1.054060,-3.768671,-0.287054,-15.230360
4,IMG_20230428_123659_jpg.rf.5e1b6c4caabe48cf360...,Sugar_400-499g_NonHalal_NonHealthy,-3.006482,-2.206970,-2.354793,-1.360836,-0.575183,-2.720716,-0.906097,-2.562416,...,-0.317981,-1.517341,-4.780657,-3.586008,-2.380914,-5.034392,3.682207,-6.363514,2.529441,-15.048306


In [32]:
main_imgs_results_small_model.shape

(4458, 60)

## All scorings from small model

In [33]:
main_imgs_results_small_model['img_type'] = "existing"
new_imgs_results_small_model['img_type'] = "new"
all_imgs_results_small_model = pd.concat([main_imgs_results_small_model, new_imgs_results_small_model], axis=0)
all_imgs_results_small_model.reset_index(drop=True, inplace=True)
all_imgs_results_small_model.head()

,filepath,label,ProductType_AdultMilk,ProductType_BabyMilkPowder,ProductType_Babyfood,ProductType_BeehoonVermicelliMeesua,ProductType_BiscuitsCrackersCookies,ProductType_Book,ProductType_BreakfastCerealsCornflakes,ProductType_CannedPacketCreamersSweet,...,Weight_500-599g,Weight_600-699g,Weight_700-799g,Weight_800-899g,Weight_900-999g,HalalStatus_Halal,HalalStatus_NonHalal,HealthStatus_Healthy,HealthStatus_NonHealthy,img_type
0,IMG_20230428_123528_jpg.rf.5687b7b914f6d9aa98c...,Sugar_400-499g_NonHalal_NonHealthy,-3.593526,-0.834642,-0.036741,-0.449187,2.034535,-2.399136,-1.877603,-3.115813,...,-1.874432,-1.630215,-3.345615,0.154984,-4.595025,-1.709256,0.742516,-4.266921,1.251087,existing
1,IMG_20230428_123522_jpg.rf.204ff37f497f2dce442...,Sugar_400-499g_NonHalal_NonHealthy,-3.660402,-2.571959,0.224866,-2.183870,-0.816285,-2.118711,-1.726907,-4.019604,...,-2.065965,-2.811384,-5.012823,0.327200,-1.995343,-1.523705,1.806258,-4.367521,0.752712,existing
2,IMG_20230428_123708_jpg.rf.141ecd0cefaea75c0b7...,Sugar_400-499g_NonHalal_NonHealthy,-0.960679,-3.024067,0.840389,-1.106379,-1.344823,-1.859851,-0.977400,-2.414862,...,0.022743,-2.701884,-5.151578,-0.955235,-1.765768,-3.311442,2.412364,-4.632944,4.555023,existing
3,IMG_20230428_123521_jpg.rf.1069b402272252862ec...,Sugar_400-499g_NonHalal_NonHealthy,-2.445158,-2.502353,-0.030090,-2.150193,-0.496334,-1.863340,-1.602406,-2.944317,...,-1.559119,-3.432280,-4.082855,-0.294293,-0.931375,-1.490215,1.054060,-3.768671,-0.287054,existing
4,IMG_20230428_123659_jpg.rf.5e1b6c4caabe48cf360...,Sugar_400-499g_NonHalal_NonHealthy,-3.006482,-2.206970,-2.354793,-1.360836,-0.575183,-2.720716,-0.906097,-2.562416,...,-0.317981,-1.517341,-4.780657,-3.586008,-2.380914,-5.034392,3.682207,-6.363514,2.529441,existing


In [34]:
all_imgs_results_small_model.tail()

,filepath,label,ProductType_AdultMilk,ProductType_BabyMilkPowder,ProductType_Babyfood,ProductType_BeehoonVermicelliMeesua,ProductType_BiscuitsCrackersCookies,ProductType_Book,ProductType_BreakfastCerealsCornflakes,ProductType_CannedPacketCreamersSweet,...,Weight_500-599g,Weight_600-699g,Weight_700-799g,Weight_800-899g,Weight_900-999g,HalalStatus_Halal,HalalStatus_NonHalal,HealthStatus_Healthy,HealthStatus_NonHealthy,img_type
4463,5151704785420_.pic.jpg,AdultMilk_1-99g_Halal_NonHealthy,9.389573,-2.244836,-1.150056,-2.540976,-0.885031,-3.015272,-3.191794,-0.882367,...,-3.866841,-2.000998,-2.607693,-1.701272,-0.689729,2.932843,-4.720117,-3.526289,3.709331,new
4464,5201704785430_.pic.jpg,AdultMilk_1-99g_Halal_NonHealthy,10.029627,-0.458718,-1.476229,-2.266092,-1.438141,-3.720444,-4.646971,-2.070412,...,-2.709237,-0.505904,-1.779749,-0.788440,-2.308589,3.605042,-4.219198,-4.332861,2.899928,new
4465,5211704785432_.pic.jpg,AdultMilk_1-99g_Halal_NonHealthy,9.932009,-0.960113,-1.285829,-2.377143,-0.638341,-3.437519,-4.333635,-2.049438,...,-3.223189,-0.972247,-1.808021,-0.821210,-1.525869,3.330086,-4.575498,-3.431279,3.549468,new
4466,5181704785427_.pic.jpg,AdultMilk_1-99g_Halal_NonHealthy,9.380651,-0.720351,-0.960074,-1.787928,-0.386909,-3.336283,-3.495619,-1.825738,...,-2.817045,-0.112533,-2.407389,-0.821929,-1.994526,3.785856,-4.707764,-3.405584,2.569691,new
4467,5141704785419_.pic.jpg,AdultMilk_1-99g_Halal_NonHealthy,7.635812,-0.835084,-0.277589,-1.891913,0.151093,-2.115914,-3.328227,-1.542094,...,-2.875587,-0.740422,-1.826304,-1.601982,-1.813279,2.628387,-3.892666,-3.142260,2.987863,new


In [35]:
all_imgs_results_small_model.shape

(4468, 60)

In [36]:
all_imgs_results_small_model.to_csv("all_imgs_results_small_model.csv", index=True)

# Bayesian model (not used)

In [37]:
prdtype_cols = [col for col in all_imgs_results_small_model.columns if col.startswith('ProductType_')]

In [38]:
all_imgs_results_small_model_prdtype = all_imgs_results_small_model[['label']+prdtype_cols]
all_imgs_results_big_model_prdtype = all_imgs_results_big_model[['label']+prdtype_cols]
all_imgs_results_small_model_prdtype = all_imgs_results_small_model_prdtype.sort_values(by='label').reset_index(drop=True)
all_imgs_results_big_model_prdtype = all_imgs_results_big_model_prdtype.sort_values(by='label').reset_index(drop=True)

In [39]:
assert (all_imgs_results_small_model_prdtype['label'][(all_imgs_results_small_model_prdtype['label'] == all_imgs_results_big_model_prdtype['label'])]).all()

ValueError: Can only compare identically-labeled Series objects

In [ ]:
all_imgs_results_small_model_prdtype['label_prdtype'] = all_imgs_results_small_model_prdtype['label'].str.split('_').str[0]
all_imgs_results_big_model_prdtype['label_prdtype'] = all_imgs_results_big_model_prdtype['label'].str.split('_').str[0]

In [ ]:
# Remove the prefix from column names
all_imgs_results_small_model_prdtype.columns = [col.replace("ProductType_", '') if col.startswith("ProductType_") else col for col in all_imgs_results_small_model_prdtype.columns]
all_imgs_results_big_model_prdtype.columns = [col.replace("ProductType_", '') if col.startswith("ProductType_") else col for col in all_imgs_results_big_model_prdtype.columns]

In [ ]:
prdtype_label_encoder = LabelEncoder()
truelabel = prdtype_label_encoder.fit_transform(all_imgs_results_big_model_prdtype['label_prdtype'])

In [ ]:
# Assuming 'category_names' is the list of unique category names in the order they appear in logitscoresA
category_names = list(all_imgs_results_small_model_prdtype['label_prdtype'].unique())
category_to_encoded = {name: prdtype_label_encoder.transform([name])[0] for name in category_names}

# Reorder columns of logitscoresA and logitscoresB to match the order of encoded labels
ordered_columns = [category_names[i] for i in prdtype_label_encoder.transform(category_names)]
logitscoresA = all_imgs_results_big_model_prdtype[ordered_columns].values
logitscoresB = all_imgs_results_small_model_prdtype[ordered_columns].values


In [ ]:
all_imgs_results_big_model.head()

In [ ]:
# big model accuracy - total
pred_big_model_prdtype = np.argmax(logitscoresA, axis=1)
sum(pred_big_model_prdtype == truelabel) / len(truelabel)

In [ ]:
# small model accuracy - total
pred_small_model_prdtype = np.argmax(logitscoresB, axis=1)
sum(pred_small_model_prdtype == truelabel) / len(truelabel)

In [ ]:
# big model accuracy - new imgs
indices = np.where(truelabel == category_to_encoded['JennyBakery'])
sum(pred_big_model_prdtype[indices] == truelabel[indices]) / len(indices[0].tolist())

In [ ]:
# small model accuracy - new imgs
indices = np.where(truelabel == category_to_encoded['JennyBakery'])
sum(pred_small_model_prdtype[indices] == truelabel[indices]) / len(indices[0].tolist())

In [ ]:
len(truelabel)

In [ ]:
logitscoresA.shape[1]

In [ ]:
ordered_columns

In [ ]:
# import pymc3 as pm
# import theano.tensor as tt
# import numpy as np
# import scipy.stats

# # Sample data setup (replace with your actual data)
# # logitscoresA and logitscoresB are matrices of logit scores for each category from classifiers A and B
# # truelabel is an already existing 1D array of integers representing the true labels
# indices = [np.random.choice(100, 3, replace=False)]  # Replace with your indices for missing data

# N = len(truelabel)
# L = logitscoresA.shape[1]
# missingidx = indices[0].tolist()  # Indices of missing data

# # Initialize truelabel_with_missing with the original truelabel and set missing indices to -1
# truelabel_with_missing = np.array(truelabel, dtype=np.int)
# truelabel_with_missing[missingidx] = -1

# # Mask the missing values
# masked_truelabel = np.ma.masked_where(truelabel_with_missing == -1, truelabel_with_missing)

# with pm.Model() as model:
#     # Priors
#     muA1 = pm.Normal('muA1', mu=0, sigma=10)
#     muA0 = pm.Normal('muA0', mu=0, sigma=10)
#     sigmaA = pm.Uniform('sigmaA', lower=0.01, upper=1.0)
#     muB1 = pm.Normal('muB1', mu=0, sigma=10)
#     muB0 = pm.Normal('muB0', mu=0, sigma=10)
#     sigmaB = pm.Uniform('sigmaB', lower=0.01, upper=1.0)
#     rho = pm.Uniform('rho', lower=-1, upper=1)
    
#     # Uniform prior over labels
#     labelprob = pm.Dirichlet('labelprob', a=tt.ones(L))

#     # Likelihood
#     muA = pm.math.switch(tt.eq(tt.arange(L), masked_truelabel[:, None]), muA1, muA0)
#     muB = pm.math.switch(tt.eq(tt.arange(L), masked_truelabel[:, None]), muB1, muB0)
    
#     logitscoresA_obs = pm.Normal('logitscoresA_obs', mu=muA, sigma=sigmaA, observed=logitscoresA)
#     logitscoresB_obs = pm.Normal('logitscoresB_obs', mu=muB + rho * (logitscoresA - muA) / sigmaA, sigma=tt.sqrt((1 - rho ** 2) * sigmaB ** 2), observed=logitscoresB)
    
#     # Define the categorical distribution for the true labels
#     truelabel_obs = pm.Categorical('truelabel_obs', p=labelprob, observed=masked_truelabel)

#     # Inference
#     trace = pm.sample(2000, tune=500, cores=1)

#     # Plotting within the model context
#     # az.plot_trace(trace)
#     # plt.show()

#     # Posterior predictive checks
#     ppc = pm.sample_posterior_predictive(trace, var_names=['truelabel_obs'])

# # Process the posterior predictive checks for missing indices
# infer_labels = []
# for idx in missingidx:
#     label_samples = ppc['truelabel_obs'][:, idx]
#     inferred_label = scipy.stats.mode(label_samples).mode[0]
#     infer_labels.append(inferred_label)

# # Output the inferred labels for missing indices
# print("Inferred labels for missing indices:", infer_labels)

In [ ]:
np.unique(infer_labels)

In [ ]:
~np.isin(np.arange(N), missingidx)

In [ ]:
np.array(truelabel, dtype=np.int)[missingidx]

In [ ]:
pred_small_model_prdtype[missingidx]

In [ ]:
pred_big_model_prdtype[missingidx]

In [ ]:
missingidx

In [ ]:
ppc['truelabel_obs'].shape